In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import os 
import glob
import logging
logging.basicConfig(level=logging.DEBUG)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:

class HandGestureDataset(Dataset):
    def __init__(self, data_dir, samples_per_second):
        self.data_files = glob.glob(os.path.join(data_dir, "*.txt"))
        self.samples_per_second = samples_per_second
        self.data = []
        self.labels = []
        self.char_to_idx = {}
        self.idx_to_char = {}
        
        for file in self.data_files:
            with open(file, 'r') as f:
                word_data = []
                for line in f:
                    values = line.strip().split()
                    if len(values) != 19:  # 18 inputs + 1 output
                        continue
                    word_data.append([float(x) for x in values[:18]])
                    if len(word_data) == samples_per_second:
                        self.data.append(word_data)
                        char = values[18]
                        if char not in self.char_to_idx:
                            idx = len(self.char_to_idx)
                            self.char_to_idx[char] = idx
                            self.idx_to_char[idx] = char
                        self.labels.append(self.char_to_idx[char])
                        word_data = []
        
        self.data = torch.FloatTensor(self.data)  # Shape: (num_words, samples_per_second, 18)
        self.data = self.data.view(self.data.size(0), self.data.size(1), 6, 3)  # Reshape to (num_words, samples_per_second, 6, 3)
        self.labels = torch.LongTensor(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

    def get_num_classes(self):
        return len(self.char_to_idx)

class HandGestureModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=64, num_layers=2, num_classes=10):
        super(HandGestureModel, self).__init__()
        self.lstm = nn.LSTM(input_size * 6, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        batch_size, time_steps, sensors, angles = x.size()
        x = x.view(batch_size, time_steps, sensors * angles)
        
        _, (h_n, _) = self.lstm(x)
        x = self.fc1(h_n[-1])
        x = self.relu(x)
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            logging.info(f'Train Batch {batch_idx}: Loss: {loss.item():.6f}')

def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    logging.info(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)')
    return accuracy

In [14]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    data_dir = ""  
    samples_per_second = 100 
    full_dataset = HandGestureDataset(data_dir, samples_per_second)
    
    # Split the data
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    num_classes = full_dataset.get_num_classes()
    model = HandGestureModel(num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    num_epochs = 50
    for epoch in range(num_epochs):
        logging.info(f'Epoch {epoch+1}/{num_epochs}')
        train(model, train_loader, criterion, optimizer, device)
        accuracy = test(model, test_loader, criterion, device)

    torch.save(model.state_dict(), 'hand_gesture_model.pth')
    logging.info("Model saved as hand_gesture_model.pth")

    example = torch.rand(1, samples_per_second, 6, 3).to(device)
    traced_script_module = torch.jit.trace(model, example)
    traced_script_module.save("hand_gesture_model_scripted.pt")
    logging.info("TorchScript model saved as hand_gesture_model_scripted.pt")


In [13]:
if __name__ == "__main__":
    main()

NameError: name 'glob' is not defined